In [1]:
import os.path
import torch
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import queue
import numpy as np
import multiprocessing
from utils_change1 import worker_init_fn, get_pdbs, loader_pdb, build_training_clusters, PDB_dataset, StructureDataset, StructureLoader
from model_utils import featurize, loss_smoothed, loss_nll, get_std_opt, ProteinMPNN
from tqdm import tqdm

In [2]:
def main():
    import json, time, os, sys, glob, os.path
    import shutil
    import warnings
    import numpy as np  
    import torch
    from torch import optim
    from torch.utils.data import DataLoader
    import queue
    import copy
    import torch.nn as nn
    import torch.nn.functional as F
    import random
    import os.path
    import subprocess
    import multiprocessing
    from concurrent.futures import ProcessPoolExecutor, as_completed    
    from utils_change1 import worker_init_fn, get_pdbs, loader_pdb, build_training_clusters, PDB_dataset, StructureDataset, StructureLoader
    from model_utils import featurize, loss_smoothed, loss_nll, get_std_opt, ProteinMPNN

In [3]:
    cwd = os.getcwd()
    print("Current working directory: {0}".format(cwd))
    #os.chdir('/ix1/zfeng/')
    #print(os.getcwd())

Current working directory: C:\Users\JIY106\OneDrive - University of Pittsburgh\Publications\Redesign\Github\training


In [6]:
    scaler = torch.cuda.amp.GradScaler()
    device = torch.device("cuda:0" if (torch.cuda.is_available()) else "CPU")
    print(device)
    print(torch.cuda.is_available)

cuda:0
<function is_available at 0x0000019317BC5C60>


C:\Users\JIY106\AppData\Local\Temp\ipykernel_15712\3784145566.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [7]:
    path_for_outputs = "Train_output" #Input
    base_folder = time.strftime(path_for_outputs, time.localtime())

    if base_folder[-1] != '/':
        base_folder += '/'
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    subfolders = ['model_weights']
    for subfolder in subfolders:
        if not os.path.exists(base_folder + subfolder):
            os.makedirs(base_folder + subfolder)

In [8]:
    PATH = [] #previous_checkpoint
    previous_checkpoint = ''
    logfile = base_folder + 'log1.txt'
    if not PATH:
        with open(logfile, 'w') as f:
            f.write('Epoch\tTrain\tValidation\n')

In [9]:
    path_for_training_data = "training_data" #Input
    data_path = path_for_training_data
    params = {
        "LIST"    : f"{data_path}/list_142934.csv", 
        "VAL"     : f"{data_path}/validation.txt",
        "TEST"    : f"{data_path}/test.txt",
        "DIR"     : f"{data_path}",
        "DATCUT"  : "2030-Jan-01",
        "RESCUT"  : 100000, 
        "HOMO"    : 0.70 
    }

In [10]:
    print(data_path)
    print(params)

training_data
{'LIST': 'training_data/list_142934.csv', 'VAL': 'training_data/validation.txt', 'TEST': 'training_data/test.txt', 'DIR': 'training_data', 'DATCUT': '2030-Jan-01', 'RESCUT': 100000, 'HOMO': 0.7}


In [11]:
    LOAD_PARAM = {'batch_size': 1,
                'shuffle': True,
                'pin_memory':False,
                'num_workers': 4}

In [12]:
    debug = False

In [13]:
    if debug is True:
        num_examples_per_epoch = 50
        max_protein_length = 1000
        batch_size = 1000

In [14]:
    train, valid, test = build_training_clusters(params, debug)

In [15]:
    print(len(test), len(valid), len(train))

14294 14293 114347


In [16]:
    train_set = PDB_dataset(list(train.keys()), loader_pdb, train, params)
    train_loader = torch.utils.data.DataLoader(train_set, worker_init_fn=worker_init_fn, **LOAD_PARAM)
    valid_set = PDB_dataset(list(valid.keys()), loader_pdb, valid, params)
    valid_loader = torch.utils.data.DataLoader(valid_set, worker_init_fn=worker_init_fn, **LOAD_PARAM)

In [17]:
    hidden_dim = 128 #Input, this is default
    num_encoder_layers = 3 #Input, this is default
    num_decoder_layers = 3 #Input, this is default
    num_neighbors = 48 #Input
    dropout = 0.1 #Input, this is default
    backbone_noise = 0 #Input

In [18]:
    model = ProteinMPNN(node_features=hidden_dim, 
                        edge_features=hidden_dim, 
                        hidden_dim=hidden_dim, 
                        num_encoder_layers=num_encoder_layers, 
                        num_decoder_layers=num_encoder_layers, 
                        k_neighbors=num_neighbors, 
                        dropout=dropout, 
                        augment_eps=backbone_noise)
    model.to(device)

ProteinMPNN(
  (features): ProteinFeatures(
    (embeddings): PositionalEncodings(
      (linear): Linear(in_features=66, out_features=16, bias=True)
    )
    (edge_embedding): Linear(in_features=416, out_features=128, bias=False)
    (norm_edges): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (W_e): Linear(in_features=128, out_features=128, bias=True)
  (W_s): Embedding(21, 128)
  (encoder_layers): ModuleList(
    (0-2): 3 x EncLayer(
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (dropout3): Dropout(p=0.1, inplace=False)
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (W1): Linear(in_features=384, out_features=128, bias=True)
      (W2): Linear(in_features=128, out_features=128, bias=True)
      (W3): Linear(in_features=128, out_features=128, bias=True)

In [19]:
    if PATH:
        checkpoint = torch.load(PATH)
        total_step = checkpoint['step'] #write total_step from the checkpoint
        epoch = checkpoint['epoch'] #write epoch from the checkpoint
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        total_step = 0
        epoch = 0


In [20]:
    optimizer = get_std_opt(model.parameters(), hidden_dim, total_step)

In [21]:
    if PATH:
        optimizer.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [22]:
    max_protein_length = 10000 #Input 
    num_examples_per_epoch = 1000000 #Input 
    batch_size = 2000 #Input
    num_epochs = 200 #Input 
    reload_data_every_n_epochs = 4 #Input 
    gradient_norm = -1.0 #Input
    save_model_every_n_epochs = 2 #Input
    #debug = False
    mixed_precision = True

In [ ]:
    with ProcessPoolExecutor(max_workers=12) as executor:
        q = queue.Queue(maxsize=3)
        p = queue.Queue(maxsize=3)
        for i in tqdm(range(3)):
            q.put_nowait(executor.submit(get_pdbs, train_loader, 1, max_protein_length, num_examples_per_epoch))
            p.put_nowait(executor.submit(get_pdbs, valid_loader, 1, max_protein_length, num_examples_per_epoch))

        pdb_dict_train = q.get().result()
        pdb_dict_valid = p.get().result() 
        #print("pdb_dict_train: ", pdb_dict_train) #add by myself
        #print("pdb_dict_valid: ", pdb_dict_valid) #add by myself
        dataset_train = StructureDataset(pdb_dict_train, truncate=None, max_length=max_protein_length) 
        dataset_valid = StructureDataset(pdb_dict_valid, truncate=None, max_length=max_protein_length)
        #print("dataset_train: ", dataset_train) #add by myself
        loader_train = StructureLoader(dataset_train, batch_size=batch_size)
        print("Hey phd")
        loader_valid = StructureLoader(dataset_valid, batch_size=batch_size)
        #print("loader_train: ", list(loader_train)) #add by myself
        reload_c = 0 
        for e in tqdm(range(num_epochs)):
            t0 = time.time()
            e = epoch + e
            model.train()
            train_sum, train_weights = 0., 0.
            train_acc = 0.
            #print("train_weights: ", train_weights) #add by myself
            if e % reload_data_every_n_epochs == 0:
                if reload_c != 0:
                    pdb_dict_train = q.get().result()
                    dataset_train = StructureDataset(pdb_dict_train, truncate=None, max_length=max_protein_length)
                    loader_train = StructureLoader(dataset_train, batch_size=batch_size)
                    pdb_dict_valid = p.get().result()
                    dataset_valid = StructureDataset(pdb_dict_valid, truncate=None, max_length=max_protein_length)
                    loader_valid = StructureLoader(dataset_valid, batch_size=batch_size)
                    q.put_nowait(executor.submit(get_pdbs, train_loader, 1, max_protein_length, num_examples_per_epoch))
                    p.put_nowait(executor.submit(get_pdbs, valid_loader, 1, max_protein_length, num_examples_per_epoch))
                reload_c += 1
            #print("reload_c: ", reload_c) #add by myself
            for _, batch in enumerate(loader_train):
                start_batch = time.time()
                X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
                elapsed_featurize = time.time() - start_batch
                optimizer.zero_grad()
                mask_for_loss = mask*chain_M
                
                #print("Mask:", mask) #add by myself
                #print("Chain_M:", chain_M) #add by myself
                #print("Mask_for_loss:", mask_for_loss) #add by myself
                
                if mixed_precision is True: #Input, this is default
                    with torch.cuda.amp.autocast():
                        log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                        _, loss_av_smoothed = loss_smoothed(S, log_probs, mask_for_loss)
           
                    scaler.scale(loss_av_smoothed).backward()
                     
                    if gradient_norm > 0.0:
                        total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_norm)

                    scaler.step(optimizer)
                    scaler.update()
                else:
                    log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                    _, loss_av_smoothed = loss_smoothed(S, log_probs, mask_for_loss)
                    loss_av_smoothed.backward()

                    if gradient_norm > 0.0:
                        total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_norm)

                    optimizer.step()
                
                loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)
            
                train_sum += torch.sum(loss * mask_for_loss).cpu().data.numpy()
                train_acc += torch.sum(true_false * mask_for_loss).cpu().data.numpy()
                
                #debugging
                #print("Before addition: train_weights =", train_weights) #add by myself
                #print("mask_for_loss =", mask_for_loss) #add by myself
                
                train_weights += torch.sum(mask_for_loss).cpu().data.numpy()
                
                #debugging
                #print("After addition: train_weights =", train_weights) #add by myself
                
                total_step += 1
                #print("Total_step:", total_step) #add by myself

            model.eval()
            with torch.no_grad():
                validation_sum, validation_weights = 0., 0.
                validation_acc = 0.
                for _, batch in enumerate(loader_valid):
                    X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
                    log_probs = model(X, S, mask, chain_M, residue_idx, chain_encoding_all)
                    mask_for_loss = mask*chain_M
                    loss, loss_av, true_false = loss_nll(S, log_probs, mask_for_loss)
                    
                    validation_sum += torch.sum(loss * mask_for_loss).cpu().data.numpy()
                    validation_acc += torch.sum(true_false * mask_for_loss).cpu().data.numpy()
                    validation_weights += torch.sum(mask_for_loss).cpu().data.numpy()
            
            #if train_weights == 0: #add by myself
                #print("Cannot be divided by zero") #add by myself
            #else: #add by myself
                #print("train_weights is not zero")
            train_loss = train_sum / train_weights
            train_accuracy = train_acc / train_weights
            train_perplexity = np.exp(train_loss)
            validation_loss = validation_sum / validation_weights
            validation_accuracy = validation_acc / validation_weights
            validation_perplexity = np.exp(validation_loss)
            
            train_perplexity_ = np.format_float_positional(np.float32(train_perplexity), unique=False, precision=3)     
            validation_perplexity_ = np.format_float_positional(np.float32(validation_perplexity), unique=False, precision=3)
            train_accuracy_ = np.format_float_positional(np.float32(train_accuracy), unique=False, precision=3)
            validation_accuracy_ = np.format_float_positional(np.float32(validation_accuracy), unique=False, precision=3)
    
            t1 = time.time()
            dt = np.format_float_positional(np.float32(t1-t0), unique=False, precision=1) 
            with open(logfile, 'a') as f:
                f.write(f'epoch: {e+1}, step: {total_step}, time: {dt}, train: {train_perplexity_}, valid: {validation_perplexity_}, train_acc: {train_accuracy_}, valid_acc: {validation_accuracy_}\n')
            print(f'epoch: {e+1}, step: {total_step}, time: {dt}, train: {train_perplexity_}, valid: {validation_perplexity_}, train_acc: {train_accuracy_}, valid_acc: {validation_accuracy_}')
            
            checkpoint_filename_last = base_folder+'model_weights/epoch_last.pt'.format(e+1, total_step)
            torch.save({
                        'epoch': e+1,
                        'step': total_step,
                        'num_edges' : num_neighbors,
                        'noise_level': backbone_noise,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.optimizer.state_dict(),
                        }, checkpoint_filename_last)

            if (e+1) % save_model_every_n_epochs == 0:
                checkpoint_filename = base_folder+'model_weights/epoch{}_step{}.pt'.format(e+1, total_step)
                torch.save({
                        'epoch': e+1,
                        'step': total_step,
                        'num_edges' : num_neighbors,
                        'noise_level': backbone_noise, 
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.optimizer.state_dict(),
                        }, checkpoint_filename)

100%|██████████| 3/3 [00:00<00:00, 11.74it/s]
